In [2]:
import argparse
import gc
import itertools
import pathlib

import numpy as np
import pandas as pd
import toml

In [3]:
argparser = argparse.ArgumentParser()
argparser.add_argument("--cell_type", default="all")

args = argparser.parse_args()

cell_type = args.cell_type

usage: ipykernel_launcher.py [-h] [--cell_type CELL_TYPE]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/lippincm/.local/share/jupyter/runtime/kernel-v2-165877648UqItPSb7j9.json


SystemExit: 2

/home/lippincm/miniconda3/envs/Interstellar/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
cell_type = "SHSY5Y"

In [5]:
# Parameters
aggregation = True
nomic = True

In [6]:
MODEL_TYPE = "regression"

In [7]:
file_path = pathlib.Path(
    f"../../../data/{cell_type}_preprocessed_sc_norm.parquet"
).resolve(strict=True)
data_df = pd.read_parquet(file_path)

data_df.head()

,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_Site,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,...,Nuclei_Texture_SumEntropy_CorrPM_3_01_256,Nuclei_Texture_SumVariance_CorrGasdermin_3_01_256,Nuclei_Texture_SumVariance_CorrMito_3_03_256,Nuclei_Texture_SumVariance_CorrPM_3_03_256,Metadata_Treatment,Metadata_Dose,oneb_Metadata_Treatment_Dose_Inhibitor_Dose,twob_Metadata_Treatment_Dose_Inhibitor_Dose,threeb_Metadata_Treatment_Dose_Inhibitor_Dose,fourb_Metadata_Treatment_Dose_Inhibitor_Dose
0,SH-SY5Y,B13,3765,1,6,Media_ctr,nan,None,media_ctr,0.0,...,1.851482,0.024721,0.307472,0.092086,media_ctr,0.0_None,media_ctr_0.0_0_Media_ctr_0.0_0,media_ctr_Media_ctr_nan__0.0_None,media_ctr__0.0_None__None_Media_ctr_nan,media_ctr__0.0_None__None_Media_ctr__nan
1,SH-SY5Y,B13,3765,1,6,Media_ctr,nan,None,media_ctr,0.0,...,0.897731,-0.041156,1.443262,0.009843,media_ctr,0.0_None,media_ctr_0.0_0_Media_ctr_0.0_0,media_ctr_Media_ctr_nan__0.0_None,media_ctr__0.0_None__None_Media_ctr_nan,media_ctr__0.0_None__None_Media_ctr__nan
2,SH-SY5Y,B13,3765,1,6,Media_ctr,nan,None,media_ctr,0.0,...,0.825970,-0.044386,-0.020445,0.000848,media_ctr,0.0_None,media_ctr_0.0_0_Media_ctr_0.0_0,media_ctr_Media_ctr_nan__0.0_None,media_ctr__0.0_None__None_Media_ctr_nan,media_ctr__0.0_None__None_Media_ctr__nan
3,SH-SY5Y,B13,3765,1,6,Media_ctr,nan,None,media_ctr,0.0,...,0.459088,-0.060584,-0.069607,-0.029195,media_ctr,0.0_None,media_ctr_0.0_0_Media_ctr_0.0_0,media_ctr_Media_ctr_nan__0.0_None,media_ctr__0.0_None__None_Media_ctr_nan,media_ctr__0.0_None__None_Media_ctr__nan
4,SH-SY5Y,B13,3765,1,6,Media_ctr,nan,None,media_ctr,0.0,...,-0.488278,-0.052383,0.006078,-0.060355,media_ctr,0.0_None,media_ctr_0.0_0_Media_ctr_0.0_0,media_ctr_Media_ctr_nan__0.0_None,media_ctr__0.0_None__None_Media_ctr_nan,media_ctr__0.0_None__None_Media_ctr__nan


In [8]:
# subset each column that contains metadata
metadata = data_df.filter(regex="Metadata")

# get all columns that are not metadata except for metadata_Well
data = data_df.drop(metadata.columns, axis=1)

# get the metadata_Well column
metadata_well = metadata[
    ["Metadata_Well", "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
]

data_df = pd.merge(data, metadata_well, left_index=True, right_index=True)
data_df

,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,...,Nuclei_Texture_InverseDifferenceMoment_CorrER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_CorrMito_3_00_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_00_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_02_256,Nuclei_Texture_SumEntropy_CorrPM_3_01_256,Nuclei_Texture_SumVariance_CorrGasdermin_3_01_256,Nuclei_Texture_SumVariance_CorrMito_3_03_256,Nuclei_Texture_SumVariance_CorrPM_3_03_256,Metadata_Well,oneb_Metadata_Treatment_Dose_Inhibitor_Dose
0,0.806753,1.127894,-1.621196,-0.939418,-0.459493,-1.559829,-0.653718,-1.368565,-0.724462,-0.790373,...,-1.295016,-3.127056,-3.004511,-1.481117,1.851482,0.024721,0.307472,0.092086,B13,media_ctr_0.0_0_Media_ctr_0.0_0
1,-0.120360,0.935495,-0.108045,-0.212199,-0.679288,-1.403068,1.531443,-0.758046,-0.469458,-0.902025,...,-0.330951,-3.725197,-0.827474,-0.461348,0.897731,-0.041156,1.443262,0.009843,B13,media_ctr_0.0_0_Media_ctr_0.0_0
2,-0.551633,0.245350,0.015592,0.348860,-0.416346,-0.333620,1.164511,0.376164,0.077378,-0.795765,...,-1.254209,-0.691997,-1.374967,-1.337252,0.825970,-0.044386,-0.020445,0.000848,B13,media_ctr_0.0_0_Media_ctr_0.0_0
3,-0.316498,0.336804,-0.813935,0.016854,0.303227,0.310584,-0.867611,0.012472,1.214836,-0.416619,...,0.801677,-0.487180,0.050679,-0.200800,0.459088,-0.060584,-0.069607,-0.029195,B13,media_ctr_0.0_0_Media_ctr_0.0_0
4,-0.012046,-0.018079,-0.197359,-0.323890,-0.120228,0.258473,-1.462321,-0.347762,0.537756,-0.956537,...,-0.802034,-0.507851,0.419403,-0.053121,-0.488278,-0.052383,0.006078,-0.060355,B13,media_ctr_0.0_0_Media_ctr_0.0_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597897,-0.061250,-2.696045,-0.863284,-0.274327,-1.607787,-0.538936,-0.034886,-0.128371,0.841032,-0.798985,...,0.456779,0.644966,-0.177565,-0.395970,1.187385,-0.042488,-0.117078,0.095893,O23,media_ctr_0.0_0_Media_ctr_0.0_0
597898,0.260396,-0.295415,-0.729197,-0.567621,-0.599516,-0.068202,1.619573,-0.602698,-0.341962,-0.777250,...,0.428563,1.114309,1.352227,0.875600,-1.099900,-0.068375,-0.124426,-0.064093,O23,media_ctr_0.0_0_Media_ctr_0.0_0
597899,-0.229473,0.202701,-0.436620,-0.089424,-0.729776,-0.627234,-1.605500,-0.177658,0.077999,-0.751063,...,0.353370,0.886794,-0.040681,-0.134595,0.077370,-0.056527,-0.119535,-0.044506,O23,media_ctr_0.0_0_Media_ctr_0.0_0
597900,-0.004208,0.800998,-0.465007,-0.331613,-0.422011,-1.003963,-1.387061,-0.231391,-1.089742,-0.950186,...,-0.479502,0.854747,0.731627,-0.213469,-0.386188,-0.051523,-0.098344,-0.038143,O23,media_ctr_0.0_0_Media_ctr_0.0_0


In [9]:
# define the list of the channels
channel_list = ["DNA", "Gasdermin", "ER", "Mito", "PM"]

In [10]:
# combiantions of channels
channel_combinations = []
for i in range(1, len(channel_list) + 1):
    tmp_list = list(itertools.combinations(channel_list, i))
    channel_combinations += tmp_list

In [11]:
# set up the LOO channel with recursion for dropping multiple channels


def channel_drop(df, channel):
    df = df.drop(df.filter(regex=channel).columns, axis=1)
    return df

In [12]:
# dictionary for each df to go into
# results_dict = {}

In [24]:
index_path = pathlib.Path(f"../indexes/{cell_type}/regression/channels").resolve()
index_path.mkdir(parents=True, exist_ok=True)

list_of_combinations = []
for i in channel_combinations:
    tmp = data_df.copy()

    if len(i) == 5:
        channel_list_index = "No Channels"
        print(i)
        for j in range(1, len(i)):
            tmp = channel_drop(tmp, i[j])
        list_of_combinations.append(channel_list_index)
        print(channel_list_index)
    elif len(i) < 5:
        for j in range(1, len(i)):
            tmp = channel_drop(tmp, i[j])
        channel_list_index = [x for x in channel_list if x in i]
        channel_list_index = "_".join(channel_list_index)
        list_of_combinations.append(channel_list_index)
        print(channel_list_index)
    new_df = pd.merge(tmp, metadata_well, left_index=True, right_index=True)
    # set file path
    file_path = pathlib.Path(
        f"../indexes/{cell_type}/regression/channels/{channel_list_index}.parquet"
    )
    # save the dataframe
    new_df.to_parquet(file_path)

DNA
Gasdermin


OSError: [Errno 28] Error writing bytes to file. Detail: [errno 28] No space left on device

In [17]:
index_list = []
for i in list_of_combinations:
    index_list.append(i + ".parquet")
# write the list to a text file
# file path
file_write_path = pathlib.Path(f"../cytokine_list/channel_splits.txt")
with open(file_write_path, "w") as f:
    for i in index_list:
        f.write("%s\n" % i)